In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
tf.__version__

'2.12.0'

In [3]:
df = pd.read_csv('Churn_Modelling.csv')
X = df.iloc[:, 3:-1].values
y = df.iloc[:, -1].values
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
ann = tf.keras.models.Sequential()

In [9]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [10]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [11]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [16]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
ann.fit(X_train,y_train, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 2s 2ms/step - loss: 0.5589 - accuracy: 0.7405
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4559 - accuracy: 0.8043
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4326 - accuracy: 0.8123
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4226 - accuracy: 0.8164
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4155 - accuracy: 0.8185
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4099 - accuracy: 0.8202
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4056 - accuracy: 0.8211
Epoch 8/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4019 - accuracy: 0.8248
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3987 - accuracy: 0.8245
Epoch 10/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3957 - accura

In [21]:
print(ann.predict(sc.transform([[1,0,0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))>0.5)

1/1 [==============================] - 0s 22ms/step
[[False]]


In [25]:
y_pred = ann.predict(X_test)
y_pred = (y_pred>0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

63/63 [==============================] - 0s 4ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [26]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
cm

array([[1519,   76],
       [ 209,  196]])

In [27]:
accuracy_score(y_test, y_pred)

0.8575